In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
from astropy.table import QTable
from astropy.table import Table
import scipy.ndimage as ndi
from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve

In [ ]:
#Creating arrays with the 8 NIRCam filters
filters = np.array([90,115,150,200,277,356,410,444])
filter_names = ['f090w','f115w','f150w','f200w','f277w','f356w','f410w','f444w']

#dictionaries for all the primer data
all_primer_data = {}

#adding all the files to the dictionaries
for i in range(len(filters)):
    primer_file = get_pkg_data_filename(f'D:\primer_cosmos_nircam_v0.5_{filter_names[i]}_30mas_sci.fits')
    all_primer_data[i] = fits.getdata(primer_file, ext=0)

In [ ]:
#reading the primer photometry catalogue
primer_phot_cat = get_pkg_data_filename('D:\PRIMER-COSMOS-v6_merged_v1.4.0.fits')
primer_phot_cat_data = fits.getdata(primer_phot_cat)#, ext=0)
primer_phot_cat_header = fits.getheader(primer_phot_cat)
tab = Table.read("D:\PRIMER-COSMOS-v6_merged_v1.4.0.fits")
RA_array = np.array(tab["RA_DETEC"])
DEC_array = np.array(tab["DEC_DETEC"])

#arrays for JWST magnitudes and fluxes
MAG_MODEL = {}
MAG_ERR_MODEL = {}
FLUX_MODEL = {}
FLUX_ERR_MODEL = {}

#Adding the photometric catalogue values to the JWST lists
for i in range(len(filters)):
    MAG_MODEL[i] = np.array(tab[f"MAG_MODEL_{filter_names[i]}"])
    MAG_ERR_MODEL[i] = np.array(tab[f"MAG_ERR_MODEL_{filter_names[i]}"])
    
    FLUX_MODEL[i] = np.array(tab[f"FLUX_MODEL_{filter_names[i]}"])
    FLUX_ERR_MODEL[i] = np.array(tab[f"FLUX_ERR_MODEL_{filter_names[i]}"])
    
#creating arrays for HST filters    
HST_filters = np.array(['F125W', 'F140W', 'F160W', 'F435W', 'F606W', 'F814W'])
HST_filters_numbers = np.array([1250, 1400, 1600, 435, 606, 814])

#arrays for HST fluxes
FLUX_MODEL_HST = {}
FLUX_MODEL_HST_ERR = {}

#Adding the photometric catalogue values to the HST lists
for i in range (len(HST_filters)):
    FLUX_MODEL_HST[i] = np.array(tab[f"FLUX_MODEL_HST-{HST_filters[i]}"])
    FLUX_MODEL_HST_ERR[i] = np.array(tab[f"FLUX_ERR_MODEL_HST-{HST_filters[i]}"])

In [ ]:
#pixel arrays corresponding to PRIMER images
DEC_values = np.linspace(2.14721639, 2.51471444, 44100)
RA_values = np.linspace(150.23463042, 150.01785458, 26000)

In [ ]:
#function for finding nearest value in array
def find_nearest(array, value):
    array = np.asarray(array)
    value = (np.abs(array - value)).argmin()
    return array[value]

In [ ]:
#reading the file with different emitters
Halpha_emitters = np.loadtxt('H-alpha emitters')
Hbeta_O3_emitters = np.loadtxt('H-beta and O3 emitters')
O2_emitters = np.loadtxt('O2 emitters')
rest_emitters = np.loadtxt('rest of emitters')

In [ ]:
#choose emission line group (possible inputs: Halpha_emitters, Hbeta_O3_emitters, O2_emitters, rest_emitters)
emitters = ?

#choose which galaxy to plot (can range from 0 to amount of galaxies within above defined emission line group)
x = ?

#Defining the galaxy
RA_coor = emitters[0][x]
DEC_coor = emitters[1][x]
classic_ID = str(emitters[2][x])[:-2]
z_phot_lephare = emitters[3][x]
z_phot_eazy = emitters[4][x]

    
#finding the galaxy in the primer data fits file
RA_val = find_nearest(RA_values, RA_coor)
DEC_val = find_nearest(DEC_values, DEC_coor)
RA_pixel = np.where(RA_values == RA_val)
DEC_pixel = np.where(DEC_values == DEC_val)

#pixel ranges of galaxy images
RA_min = RA_pixel[0][0] - 50
RA_max = RA_pixel[0][0] + 50
DEC_min = DEC_pixel[0][0] - 50
DEC_max = DEC_pixel[0][0] + 50

    
#finding the entry in primer catalogue corresponding to the emission line galaxy
dev_vector = np.sqrt((((RA_coor-RA_array))*np.cos(DEC_coor*np.pi/180))**2+(DEC_coor-DEC_array)**2)
dev_min = np.min(dev_vector)
entry = np.where(dev_vector==dev_min)[0][0]

#extracting photometry data
spectrometry = []
spectrometry_err = []
    
for i in range (len(filters)):
    spectrometry.append(MAG_MODEL[i][entry])
    spectrometry_err.append(MAG_ERR_MODEL[i][entry])


#plotting
#creating the figure
fig = plt.figure(figsize=(16,10))
pl.rcParams['font.size'] = 18   # increase fontsize to an appropriate level 
pl.rcParams['mathtext.default'] = 'regular' # display latex text with the same font as the the other text


#creating and displaying the images
ax = {}
for i in range (len(filters)):
    ax[i] = plt.subplot2grid((2, 8), (0, i))
    ax[i].imshow(all_primer_data[i][DEC_min:DEC_max,RA_min:RA_max], cmap='viridis', vmin = -0.01, vmax = 0.5, origin='lower')
    ax[i].xaxis.set_visible(False)
    ax[i].yaxis.set_visible(False)
    ax[i].text(.5,1.1, str(filters[i]/100) + '$\mu$m', horizontalalignment='center', transform=ax[i].transAxes,color='black')


#creating and displaying the spectrum
ax10 = plt.subplot2grid((2, 8), (1, 0), colspan=8)
fig.subplots_adjust(hspace = .000)
ax10.set_xlabel('Observed wavelength [$\mu$m]')
ax10.yaxis.set_visible(True)
ax10.set_ylabel('AB')
ax10.yaxis.set_visible(True)
ax10.plot(filters*0.01,spectrometry,'o',label = 'Apparent magnitudes')
ax10.errorbar(filters*0.01,spectrometry,yerr=spectrometry_err,ls='',label='Uncertainties',capsize=4)
ax10.grid(False)
ax10.invert_yaxis()
plt.legend(loc='upper left')

fig.tight_layout()
fig.suptitle("classic ID = "+f"{classic_ID}     "+"     RA = "+f"{RA_coor}     "+"     DEC = "+f"{DEC_coor}"+"\n z$_{phot(Classic, EAZY)}$ = "+f"{z_phot_eazy}     "+"     z$_{phot(Classic, LePhare)}$ = "+f"{z_phot_lephare}", size=24)
plt.show()